In [1]:
import pandas as pd
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import pipeline

In [2]:
try:
    classifier = pipeline("zero-shot-classification", device = -1)
except RuntimeError:
    print("A runtime error occurred, check if tensorflow and pytorch are correctly installed, need to be version >= 2")

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertF

In [3]:
df_name  = pd.read_csv(r'csv_name')

FileNotFoundError: [Errno 2] No such file or directory: 'csv_name'

In [ ]:
# Dataprocessing should be inputted here, see Clara's code

In [4]:
candidate_labels = ['racist', 'sexist', 'hatespeech', 'neutral']
candidate_results = [0, 0, 0, 0]

for sent in tqdm(df_name['name_of_tweets_column'].values):
    res = classifier(sent, candidate_labels)

    if res['labels'][0] == 'racist' and res['scores'][0] > 0.5:
        candidate_results[0] = candidate_results[0] + 1
    if res['labels'][0] == 'sexist' and res['scores'][0] > 0.5:
        candidate_results[1] = candidate_results[1] + 1
    if res['labels'][0] == 'hatespeech' and res['scores'][0] > 0.5:
        candidate_results[2] = candidate_results[2] + 1
    if res['labels'][0] == 'neutral' and res['scores'][0] > 0.5:
        candidate_results[3] = candidate_results[3] + 1

    if res['scores'][0] > 0.5:
        print(sent)
        print(res['labels'])
        print(res['scores'])
        print('\n')

print(candidate_results)

NameError: name 'df_name' is not defined

In [ ]:
data = {'labels': candidate_labels, 'values': candidate_results}

df_frequency = pd.DataFrame(data, columns=['labels', 'values'])

df_frequency.head()

In [ ]:
sns.barplot(data = df_frequency, x = 'lables', y = 'values')